In [1]:
import numpy as np
import autoassigner
from matcher.solvers.simple_solver import SimpleSolver
from matcher.solvers.maximin_solver import MaxiMinSolver


## Examples

In [2]:
avg_est = lambda x: x
mle_est = lambda x: 1./(1-x) if x < 1 else 1e6

### Example 1:
Comparison between **Ivan's autoassigner (PeerReview4All)**, the **simple solver** (global maximizer; used by TPMS and ORPMS currently), and the **PeerReview4All subroutine** (maximizes the minimum scores per paper).

In [3]:
similarity0 = np.matrix([[1., 1., 1.], 
                         [0., 0., 1./4], 
                         [1./4, 1./4, 1./2]])

cost0 = np.asarray(-1 * similarity0)
constraint0 = np.asarray(0 * similarity0)

papers_per_reviewer0 = 1
reviewers_per_paper0 = 1
num_reviewers0 = 3

In [4]:
a = autoassigner.auto_assigner(
    similarity0, 
    papers_per_reviewer0, 
    reviewers_per_paper0, 
    avg_est
)

a.fair_assignment('fast')
print('fairness of the resulting assignment:', "%.2f" % a.best_quality)

# a.fa is the FA ("fair assignment") of solver a. 
# it is a dict keyed on paper index. each value is a list of reviewer indices.
a.fa

Academic license - for non-commercial use only
fairness of the resulting assignment: 0.25


{0: [2], 1: [0], 2: [1]}

In [5]:
b = autoassigner.auto_assigner(
    similarity0, 
    papers_per_reviewer0, 
    reviewers_per_paper0, 
    avg_est
)

b.fair_assignment('ortools')
print('fairness of the resulting assignment:', "%.2f" % a.best_quality)

# a.fa is the FA ("fair assignment") of solver a. 
# it is a dict keyed on paper index. each value is a list of reviewer indices.
b.fa

fairness of the resulting assignment: 0.25


{0: [1], 1: [2], 2: [0]}

In [6]:
a.fa

{0: [2], 1: [0], 2: [1]}

In [7]:
def translate_openreview_solution(solution):
    openreview_assignment = {}
    for idx, row in enumerate(solution):
        openreview_assignment[idx] = []
        for cidx, cell in enumerate(row):
            if cell == 1.0:
                openreview_assignment[idx].append(int(cidx))

    return openreview_assignment


In [8]:
simple_solver = SimpleSolver(
    num_reviewers0*[papers_per_reviewer0], 
    num_reviewers0*[papers_per_reviewer0], 
    num_reviewers0*[reviewers_per_paper0], 
    cost0,
    constraint0
)

translate_openreview_solution(simple_solver.solve())

{0: [0], 1: [2], 2: [1]}

In [9]:
maximin_solver = MaxiMinSolver(
    num_reviewers0*[papers_per_reviewer0], 
    num_reviewers0*[reviewers_per_paper0], 
    cost0,
    constraint0
)
translate_openreview_solution(maximin_solver.solve())

{0: [1], 1: [2], 2: [0]}

### Example 2

In [12]:
similarity1 = np.matrix([[1., 1., 1., 1., 0., 0.], [0., 0., 0., 0., 1./3, 1./3], 
                        [0., 0., 0., 0., 1./3., 1./3], [0., 0., 0., 0., 1./3., 1./3],
                        [1./2, 1./2, 1./2, 1./2, 1./2., 1./2.], [1./2, 1./2, 1./2, 1./2, 1./2., 1./2.]])

cost1 = np.asarray(-1 * similarity1)
constraint1 = np.asarray(0 * similarity1)

papers_per_reviewer1 = 2
reviewers_per_paper1 = 2
num_reviewers1 = 6

In [13]:
a1 = autoassigner.auto_assigner(
    similarity1, 
    papers_per_reviewer1, 
    reviewers_per_paper1, 
    mle_est
)
a1.fair_assignment('full')
print('fairness of the resulting assignment:', "%.2f" % a1.best_quality)

fairness of the resulting assignment: 3.00


## OR Tools testing

In [ ]:
# """From Bradley, Hax, and Magnanti, 'Applied Mathematical Programming', figure 8.1."""

from __future__ import print_function
from ortools.graph import pywrapgraph

"""MinCostFlow simple interface example."""

# Define four parallel arrays: start_nodes, end_nodes, capacities, and unit costs
# between each pair. For instance, the arc from node 0 to node 1 has a
# capacity of 15 and a unit cost of 4.

start_nodes = [ 0, 0,  1, 1,  1,  2, 2,  3, 4]
end_nodes   = [ 1, 2,  2, 3,  4,  3, 4,  4, 2]
capacities  = [15, 8, 20, 4, 10, 15, 4, 20, 5]
# capacities  = [ 1, 1,  1, 1,  1,  1, 1,  1, 1]
unit_costs  = [ 4, 4,  2, 2,  6,  1, 3,  2, 3]

# Define an array of supplies at each node.

supplies = [20, 0, 0, -5, -15]


# Instantiate a SimpleMinCostFlow solver.
min_cost_flow = pywrapgraph.SimpleMinCostFlow()

# Add each arc.
for i in range(0, len(start_nodes)):
    min_cost_flow.AddArcWithCapacityAndUnitCost(start_nodes[i], end_nodes[i],
                                            capacities[i], unit_costs[i])

# Add node supplies.

for i in range(0, len(supplies)):
    min_cost_flow.SetNodeSupply(i, supplies[i])


# Find the minimum cost flow between node 0 and node 4.
if min_cost_flow.Solve() == min_cost_flow.OPTIMAL:
    print('Minimum cost:', min_cost_flow.OptimalCost())
    print('')
    print('  Arc    Flow / Capacity  Cost')
    total_flow = 0
    for i in range(min_cost_flow.NumArcs()):
        cost = min_cost_flow.Flow(i) * min_cost_flow.UnitCost(i)
        print('%1s -> %1s   %3s  / %3s       %3s' % (
            min_cost_flow.Tail(i),
            min_cost_flow.Head(i),
            min_cost_flow.Flow(i),
            min_cost_flow.Capacity(i),
            cost))
        total_flow += min_cost_flow.Flow(i)
else:
    print('There was an issue with the min cost flow input.')



In [ ]:
min_cost_flow.Solve()

In [ ]:
sum(supplies)

In [ ]:
min_cost_flow.NOT_SOLVED